## How to?
1. Create the credential for youtube v3 in your project
2. download the credentials 
3. Add test user 
4. Run code snippets and authenticate in browser

In [26]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.traxsource.com/genre/4/house"
#URL = "https://www.traxsource.com/top/tracks"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [45]:
page = requests.get("https://www.traxsource.com/genre/4/house/top")
soup = BeautifulSoup(page.content, "html.parser")

top100 = soup.find_all("div", {"class": "trk-row"})

In [77]:
title = top100[2].find("div", {"class": "tnum-pos"}).text.replace("\n", "")
title


'2'

In [82]:
def get_top_10(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    top10 = soup.find_all("div", {"class": "top-item"})

    top10_title = []
    top10_artist = []

    for track in top10:

        track_node = track

        # Get position of the track
        position = track_node.findAll("div", {"class": "ttib position"})[0].getText()
        position = position.replace('\n', '').strip()

        # Get track infos
        info = track_node.findAll("div", {"class": "ttib info"})[0]
        artist = info.find("a", class_='com-artists').text
        title = info.find("a", class_='com-title').text

        # Add infos
        top10_title.append(title)
        top10_artist.append(artist)

    return top10_title, top10_artist

def get_top_100(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")

    top100 = soup.find_all("div", {"class": "trk-row"})

    top100_title = []
    top100_artist = []

    for track in top100[1:]:

        # Get position of the track
        pos = track.find("div", {"class": "tnum-pos"}).text.replace("\n", "").strip()

        # Get artist infos
        artists = track.find("div", {"class": "artists"}).findChildren("a")
        artists = [art.text for art in artists]
        full_artist = " ".join(artists)

        # Get title info
        title = track.find("div", {"class": "title"}).findChildren("a")[0].text

        # Add infos
        top100_title.append(title)
        top100_artist.append(full_artist)

    return top100_title, top100_artist

In [27]:
top10 = soup.find_all("div", {"class": "top-item"})

for track in top10:

    track_node = track

    # Get position of the track
    position = track_node.findAll("div", {"class": "ttib position"})[0].getText()
    position = position.replace('\n', '').strip()

    # Get track infos
    info = track_node.findAll("div", {"class": "ttib info"})[0]
    artist = info.find("a", class_='com-artists').text
    title = info.find("a", class_='com-title').text
    label = info.find("a", class_='com-label').text

    # Print infos
    print('Position: ', position)
    print('Name:     ', title)
    print('Artist:   ', artist)
    print('Label:    ', label)
    print('')

Position:  1
Name:      Do What You Wanna Do (Extended Mix)
Artist:    The Shapeshifters
Label:     Pleased As Punch

Position:  2
Name:      I Thought It Was You (Extended Mix)
Artist:    David Penn
Label:     Central Station Records

Position:  3
Name:      Can U Feel It (2023 Remaster)
Artist:    Jay Vegas
Label:     Hot Stuff

Position:  4
Name:      Don't Pay Me No Mind
Artist:    Mike Dunn
Label:     Nu Groove Records

Position:  5
Name:      Runaway Love (Michael Gray Dub)
Artist:    Bobby D'Ambrosio
Label:     Soulfuric

Position:  6
Name:      Frequency
Artist:    Grant Nelson
Label:     Swing City

Position:  7
Name:      Serious (Wayne Soul Avengerz & Odyssey Inc. Extended Remix)
Artist:    Donna Allen
Label:     High Fashion Music

Position:  8
Name:      Living My Life (Golden) (Extended Mix)
Artist:    Higgo
Label:     Helix Records

Position:  9
Name:      Do You Feel The Same? (Luke Solomon Tribute Remix)
Artist:    Hercules & Love Affair
Label:     Defected

Position: 

In [88]:
import os
import pickle
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

# Helper function to authenticate with the YouTube API
def get_authenticated_service():
    # Define the necessary scopes required for the YouTube Data API v3
    scopes = ["https://www.googleapis.com/auth/youtube"]

    # Load or create credentials file (store it in a file for future use)
    credentials_file = "youtube_credentials.pickle"

    if os.path.exists(credentials_file):
        with open(credentials_file, "rb") as token:
            credentials = pickle.load(token)
    else:
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
            "secrets_file.json", scopes
        )
        credentials = flow.run_local_server(port=0)

        with open(credentials_file, "wb") as token:
            pickle.dump(credentials, token)

    # Create and return the authenticated YouTube Data API service
    return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)


youtube = get_authenticated_service()

In [89]:
def get_video_info(video_id):
    try:
        # Call the videos().list() method to get video information
        video_info = youtube.videos().list(
            part="snippet,statistics",
            id=video_id,
        ).execute()


        # Extract the relevant information from the response
        video_title = video_info['items'][0]['snippet']['title']
        video_description = video_info['items'][0]['snippet']['description']
        view_count = video_info['items'][0]['statistics']['viewCount']
        like_count = video_info['items'][0]['statistics']['likeCount']

        # Print or return the information as needed
        print(f"Video Title: {video_title}")
        print(f"Video Description: {video_description}")
        print(f"Views: {view_count}")
        print(f"Likes: {like_count}")

    except Exception as e:
        print(f"Error: {e}")

# Replace 'YOUR_VIDEO_ID' with the ID of the video you want to get information about
video_id = "bHeA_5yKslk"
get_video_info(video_id)

Video Title: VLNT - Minimal House Mix (Let's House Party | SG) #18
Video Description: instagram:

https://www.instagram.com/ciuca_valentine/

Contact: ciucavalentin2000@gmail.com
Views: 421
Likes: 14


In [90]:
def get_song_id(name, max_results=1):
    try:
        # Call the search().list() method to search for videos
        search_response = youtube.search().list(
            q=name,
            part="snippet",
            type="video",
            maxResults=max_results
        ).execute()

        # Extract video ID of the first
        for search_result in search_response.get("items", []):
            video_id = search_result["id"]["videoId"]
            video_title = search_result["snippet"]["title"]
            video_description = search_result["snippet"]["description"]
        
        return video_id

    except Exception as e:
        print(f"Error: {e}")

search_query = "Do What You Wanna Do (Extended Mix)"

get_song_id(search_query)


'opFJuu2zpyU'

In [91]:
def get_playlist_ids():
    try:
        # Call the playlists().list() method to get playlists associated with the authenticated user
        playlist_response = youtube.playlists().list(
            part="id,snippet",
            mine=True,
            maxResults=50  # Set the maximum number of playlists to retrieve (default is 5)
        ).execute()

        # Extract and display the playlist IDs
        for playlist in playlist_response.get("items", []):
            playlist_id = playlist["id"]
            playlist_title = playlist["snippet"]["title"]
            print(f"Playlist ID: {playlist_id}")
            print(f"Playlist Title: {playlist_title}")
            print()

    except Exception as e:
        print(f"Error: {e}")

get_playlist_ids()

Playlist ID: PL1sgmjVOC2BA96I-pAz0IRw9sVZgbDYHK
Playlist Title: Traxsource Top 100 House

Playlist ID: PL1sgmjVOC2BDwXHkTBeD1ISjstJIJXAeC
Playlist Title: Traxsource Top 10



In [84]:
def create_playlist(playlist_title, playlist_description):
    try:
        # Call the playlists().insert() method to create a new playlist
        playlist_response = youtube.playlists().insert(
            part="snippet,status",
            body={
                "snippet": {
                    "title": playlist_title,
                    "description": playlist_description,
                },
                "status": {
                    "privacyStatus": "public",  # Set the privacy status ("public", "private", or "unlisted")
                }
            }
        ).execute()

        # Extract and display the playlist ID of the newly created playlist
        playlist_id = playlist_response["id"]
        print(f"New playlist created with ID: {playlist_id}")

    except Exception as e:
        print(f"Error: {e}")

create_playlist(playlist_title="Traxsource Top 100 House", playlist_description="The current Top 100 Charts on Traxsource for the genre House")

New playlist created with ID: PL1sgmjVOC2BA96I-pAz0IRw9sVZgbDYHK


In [85]:
# Function to add a video to a playlist
def add_video_to_playlist(playlist_id, video_id):

    # Add the video to the playlist
    request = youtube.playlistItems().insert(
        part="snippet",
        body={
            "snippet": {
                "playlistId": playlist_id,
                "resourceId": {"kind": "youtube#video", "videoId": video_id},
            }
        },
    )

    response = request.execute()

    #print(f"Video {video_id} added to the playlist {playlist_id}!")



In [1]:
from traxsource.traxsource import get_top_10

minimal_url = 'https://www.traxsource.com/genre/16/minimal-deep-tech'

playlist_id = "PL1sgmjVOC2BA96I-pAz0IRw9sVZgbDYHK"

top10_title, top10_artist = get_top_10(URL=minimal_url)

for title, artist in zip(top10_title, top10_artist):
    # Get video id of top result
    query = f"{artist} - {title}"
    video_id = get_song_id(name=query)

    # Add to the playlist
    add_video_to_playlist(playlist_id, video_id)

    print(f"{title} by {artist} was added.")



(['Disco Party Baby (Original Mix)',
  'Hypnotized (Original Mix)',
  'Shake For Me (Original Mix)',
  'High Rise (Original Mix)',
  'Pay Attention (SIREN & PREACH Mix)',
  'Streets (Dachshund Remix)',
  'Red Lights (Extended Mix)',
  'Jiggy',
  'Messan (Original Mix)',
  'Chess (Original Mix)'],
 ['Vampire Sex',
  'Audiojack',
  'Paul Mondot',
  'Secnd',
  'Soul Vision',
  'Greta Levska',
  'Brock Edwards',
  'Blair Suarez',
  'Jansons',
  'Josh Butler'])

In [87]:
playlist_id = "PL1sgmjVOC2BA96I-pAz0IRw9sVZgbDYHK"

top100_title, top100_artist = get_top_100(URL="https://www.traxsource.com/genre/4/house/top")

for title, artist in zip(top100_title, top100_artist):
    query = f"{artist} - {title}"
    video_id = get_song_id(name=query)

    add_video_to_playlist(playlist_id, video_id)

Error: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/search?q=The+Shapeshifters+-+Do+What+You+Wanna+Do&part=snippet&type=video&maxResults=1&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">


HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/playlistItems?part=snippet&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "[{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]">